In [1]:
import os
import glob

import numpy as np

from medpy.metric import binary

from PIL import Image

import imageio

In [100]:
# task = '_betnorm' # _gmnorm _ventnorm _wmnorm _betnorm _csfnorm
# gt_dir = '/data/ychan019/experiments/radiaide/radiaide/code/inference/test_kcl_p2p_seg_alt' + task + '/gt/'
# syn_dir = '/data/ychan019/experiments/radiaide/radiaide/code/inference/test_kcl_p2p_seg_alt' + task + '/mask/' # sub-0032_20.png
task = 'csf_alt'
gt_dir = '/data/data_repo/neuro_img/anat_brain_img/datasets/CERMEP-IDB-MRXFDG/processed/yihao/labels_new_test/' + task
syn_dir = '/data/ychan019/experiments/radiaide/Pytorch-UNet-master/data/mask_' + task

In [101]:
all_filepaths_gt = sorted(glob.glob(gt_dir + "/*.*"))
all_filepaths_syn = sorted(glob.glob(syn_dir + "/*.*"))

In [102]:
from collections import defaultdict

prev_subject_id = 'sub-0028'
hist_dc = defaultdict(list)
hist_jc = defaultdict(list)

fp = []
missed = []

for subj_path_gt, subj_path_syn in zip(all_filepaths_gt, all_filepaths_syn):
    
    # gt = np.array(Image.open(subj_path_gt).convert("L"))
    gt = np.array(imageio.imread(subj_path_gt)) # filename
    syn = np.array(Image.open(subj_path_syn).convert("L"))
    
    if np.sum(gt) == 0: # and np.sum(syn) == 0: # if no incl syn this will miss out on false positive
        if np.sum(syn) != 0:
            fp.append((subj_path_gt, subj_path_syn))
        continue
    
    cur_subject_id_full = subj_path_syn.split('/')[-1]
    cur_subject_id = cur_subject_id_full.split('_')[0]
    
    dc = binary.dc(gt, syn) # https://loli.github.io/medpy/generated/medpy.metric.binary.dc.html#medpy.metric.binary.dc
    jc = binary.jc(gt, syn) # https://loli.github.io/medpy/generated/medpy.metric.binary.jc.html#medpy.metric.binary.jc
    print(cur_subject_id_full, dc, jc)
    
    if cur_subject_id != prev_subject_id:
        print()
    
    prev_subject_id = cur_subject_id
    
    if dc != 0 or jc != 0:
        hist_dc[cur_subject_id].append(dc)
        hist_jc[cur_subject_id].append(jc)
    else:
        missed.append((subj_path_gt, subj_path_syn))

sub-0028_100.png 0.5413958745408307 0.37117396358000776
sub-0028_101.png 0.5571288377961587 0.3861251457442674
sub-0028_102.png 0.5686918445539135 0.39732313575525813
sub-0028_103.png 0.5874331550802139 0.4158621995078554
sub-0028_104.png 0.5910509885535901 0.4194977843426883
sub-0028_105.png 0.602874432677761 0.431510557661072
sub-0028_106.png 0.6174762081324929 0.4466297157160737
sub-0028_107.png 0.6204131227217496 0.449709353531795
sub-0028_108.png 0.6061486322924231 0.43487321986800975
sub-0028_109.png 0.6056691783279512 0.4343798250128667
sub-0028_110.png 0.6072864916016087 0.43604552403601154
sub-0028_111.png 0.6369560158249942 0.4673040805873314
sub-0028_112.png 0.6503237742830712 0.4818368745716244
sub-0028_113.png 0.6453636997443644 0.4764110482072397
sub-0028_114.png 0.6228226093138998 0.4522457408363449
sub-0028_115.png 0.6242516727315413 0.4537542662116041
sub-0028_116.png 0.619197942242488 0.44843353090601185
sub-0028_117.png 0.599930078079478 0.4285000832362244
sub-0028_1

In [103]:
all_dc = []

for k,v in hist_dc.items():
    
    all_dc.extend(v)
    
    sub_scores = np.array(v)
    print(k, sub_scores.mean(), sub_scores.std(), sub_scores.min(), sub_scores.max())


sub-0028 0.5210795500152003 0.11553355770868907 0.03558718861209965 0.6728121927236972
sub-0029 0.46413258191787093 0.11748882284378952 0.012987012987012988 0.616526712912255
sub-0030 0.43660489502365185 0.11214889444985818 0.037037037037037035 0.5311449475973897
sub-0031 0.47269828932733926 0.10444440225670866 0.01092896174863388 0.5954721477509681
sub-0032 0.4531740929925394 0.13110778544074086 0.046610169491525424 0.578939681612385
sub-0033 0.4892376763744785 0.15695579862038636 0.007692307692307693 0.655185513486134
sub-0034 0.4691016268049892 0.1147569047222892 0.009324009324009324 0.5909224874742116
sub-0035 0.5200101212118097 0.12362804928359689 0.03870967741935484 0.650478069509789
sub-0036 0.47355532461340116 0.1144720667179299 0.02531645569620253 0.5856034089179729
sub-0037 0.4917417679803611 0.11939493346421064 0.005747126436781609 0.6497515040544075


In [104]:
all_jc = []

for k,v in hist_jc.items():

    all_jc.extend(v)
    
    sub_scores = np.array(v)
    print(k, sub_scores.mean(), sub_scores.std(), sub_scores.min(), sub_scores.max())


sub-0028 0.35973414857723757 0.09523963300406432 0.018115942028985508 0.5069457306908687
sub-0029 0.3090498619121538 0.09009306727484376 0.006535947712418301 0.4456368754398311
sub-0030 0.28496478139567083 0.07961156722633571 0.018867924528301886 0.36160473882606353
sub-0031 0.31492976830292013 0.07977734738083361 0.005494505494505495 0.423966065747614
sub-0032 0.30102889597955457 0.09544080887780461 0.02386117136659436 0.4073997944501542
sub-0033 0.33621613695557245 0.11981849664823005 0.003861003861003861 0.4871939736346516
sub-0034 0.3127625415470484 0.0850346907839088 0.00468384074941452 0.41936833298473125
sub-0035 0.35978239860364397 0.10135152070551653 0.019736842105263157 0.4820062977957715
sub-0036 0.31670428639139453 0.08666279105145931 0.01282051282051282 0.4140305573488272
sub-0037 0.33343909960303936 0.09411348858237635 0.002881844380403458 0.4812088337853545


In [105]:
all_dc = np.array(all_dc)
all_jc = np.array(all_jc)

In [106]:
all_dc.mean(), all_dc.std()

(0.4792449140571837, 0.12448897071650096)

In [107]:
all_jc.mean(), all_jc.std()

(0.32296554764356006, 0.09625365607200768)

In [108]:
len(fp)

24

In [109]:
len(missed)

71

In [110]:
gt

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [111]:
syn

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [112]:
np.unique(gt)

array([0, 1], dtype=uint8)

In [113]:
np.unique(syn)

array([  0, 127], dtype=uint8)

In [114]:
np.sum(gt), np.sum(syn)/127

(3161, 3178.0)